# Single-Cell RNA-seq Cell Annotation Pipeline

This notebook provides an interactive walkthrough of the complete cell annotation pipeline, from raw CellBender output to annotated cell types.

**Authors:** Tsai Lab  
**Last Updated:** 2025-01-11  
**Version:** 2.0 (Corrected to match original pipeline)

⚠️ **Important**: This notebook uses the exact calculation methods from the original pipeline to ensure reproducible results.

---

## Table of Contents

1. [Setup & Installation](#setup)
2. [Parameter Configuration](#parameters)
3. [Stage 1: Data Loading & Integration](#stage1)
4. [Stage 2: QC Metrics Calculation & Visualization](#stage2)
5. [Stage 3: Doublet Detection](#stage3)
6. [Stage 4: Cell & Gene Filtering](#stage4)
7. [Stage 5: Normalization & Scaling](#stage5)
8. [Stage 6: PCA, UMAP & Clustering](#stage6)
9. [Stage 7: Marker Gene Analysis](#stage7)
10. [Stage 8: Cell Type Annotation](#stage8)
11. [Stage 9: Reclustering & Export](#stage9)
12. [Summary & Next Steps](#summary)

---

## 1. Setup & Installation

Install required packages and set up the environment.

In [ ]:
# Install required packages
!pip install -q scanpy anndata scrublet matplotlib seaborn scikit-learn pandas numpy h5py scipy

# Import libraries
import warnings
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import scanpy as sc
import pandas as pd
import numpy as np
import h5py
from scipy import sparse
import anndata
from pathlib import Path
from IPython.display import display, HTML, Markdown
import scrublet as scr

# Configure settings
warnings.filterwarnings('ignore')
sc.settings.verbosity = 3
sc.settings.set_figure_params(dpi=80, facecolor='white')
matplotlib.rcParams['figure.figsize'] = (8, 6)

print("✓ Setup complete!")
print(f"Scanpy version: {sc.__version__}")

### Mount Google Drive (if using Colab)

Uncomment and run if you need to access data from Google Drive.

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
# BASE_PATH = "/content/drive/MyDrive/your_data_path/"

## 2. Parameter Configuration

This section centralizes all tunable parameters for the pipeline. Adjust these based on your data characteristics and analysis goals.

### 🎯 Quick Start Modes

Choose a preset or customize individual parameters below.

In [ ]:
# ============================================================================
# PRESET CONFIGURATIONS
# ============================================================================

PRESETS = {
    'default': {
        'name': 'Default (Balanced)',
        'description': 'Standard parameters suitable for most datasets',
    },
    'stringent': {
        'name': 'Stringent QC',
        'description': 'Stricter filtering for high-quality cells only',
    },
    'permissive': {
        'name': 'Permissive QC',
        'description': 'More lenient filtering to retain more cells',
    },
}

# Select your preset here
SELECTED_PRESET = 'default'  # Options: 'default', 'stringent', 'permissive'

print(f"Selected preset: {PRESETS[SELECTED_PRESET]['name']}")
print(f"Description: {PRESETS[SELECTED_PRESET]['description']}")

In [ ]:
# ============================================================================
# DATA LOADING PARAMETERS
# ============================================================================

# Path to CellBender output files
BASE_PATH = "/path/to/your/data/"  # 🔧 UPDATE THIS PATH

# Sample identifiers
# Example: [f"D25-{i}" for i in range(2675, 2691)] generates D25-2675, D25-2676, ..., D25-2690
SAMPLE_NAMES = [f"D25-{i}" for i in range(2675, 2691)]  # 🔧 CUSTOMIZE YOUR SAMPLES

# CellBender output filename pattern
CUSTOM_NAME = "_processed_feature_bc_matrix_filtered.h5"  # 🔧 UPDATE IF DIFFERENT

# Output directory for plots and results
PLOTS_DIR = Path("plots")
PLOTS_DIR.mkdir(exist_ok=True)

print(f"Data path: {BASE_PATH}")
print(f"Number of samples: {len(SAMPLE_NAMES)}")
print(f"Output directory: {PLOTS_DIR}")

In [ ]:
# ============================================================================
# QC FILTERING PARAMETERS
# ============================================================================

# Define parameter sets for each preset
QC_PRESETS = {
    'default': {
        'min_genes': 200,
        'max_genes': 8000,
        'min_counts': 1000,
        'max_counts': 50000,
        'max_mt_pct': 10,
        'max_ribo_pct': None,
    },
    'stringent': {
        'min_genes': 500,
        'max_genes': 6000,
        'min_counts': 1500,
        'max_counts': 40000,
        'max_mt_pct': 5,
        'max_ribo_pct': None,
    },
    'permissive': {
        'min_genes': 100,
        'max_genes': 10000,
        'min_counts': 500,
        'max_counts': 60000,
        'max_mt_pct': 15,
        'max_ribo_pct': None,
    },
}

# Load parameters based on selected preset
CELL_FILTERS = QC_PRESETS[SELECTED_PRESET]

# 🔧 OPTIONAL: Override specific parameters here
# Uncomment and modify any parameter you want to customize:
# CELL_FILTERS['min_genes'] = 300
# CELL_FILTERS['max_mt_pct'] = 8

# Gene-level filters
GENE_FILTERS = {
    'min_cells': 10,  # 🔧 Minimum cells expressing a gene
}

# Mitochondrial and ribosomal gene patterns
GENE_PATTERNS = {
    'mt_pattern': 'mt-',     # 🔧 Use 'MT-' for human, 'mt-' for mouse
    'ribo_pattern': r'^Rp[sl]',  # Ribosomal protein genes
}

# Display current settings
print("Cell-level QC filters:")
for key, value in CELL_FILTERS.items():
    print(f"  {key}: {value}")

print("\nGene-level filters:")
for key, value in GENE_FILTERS.items():
    print(f"  {key}: {value}")

In [ ]:
# ============================================================================
# DOUBLET DETECTION PARAMETERS
# ============================================================================

DOUBLET_PARAMS = {
    'expected_doublet_rate': 0.10,  # 🔧 10% expected doublet rate (platform-dependent)
    'manual_threshold': 0.35,       # 🔧 Score threshold for doublet classification
    'min_counts': 2,                # Minimum counts for Scrublet filtering
    'min_cells': 3,                 # Minimum cells for Scrublet filtering
    'min_gene_variability_pctl': 85,  # Gene variability percentile
    'n_prin_comps': 30,             # Number of principal components
}

# 💡 Tips for tuning:
# - expected_doublet_rate: 0.06 for 10x v3, 0.08-0.10 for high-throughput
# - manual_threshold: Lower (0.25-0.30) for stricter removal, Higher (0.40-0.45) for permissive

print("Doublet detection parameters:")
for key, value in DOUBLET_PARAMS.items():
    print(f"  {key}: {value}")

In [ ]:
# ============================================================================
# DIMENSIONALITY REDUCTION & CLUSTERING PARAMETERS
# ============================================================================

# PCA parameters
N_PCS = 15  # 🔧 Number of principal components (check elbow plot to adjust)
N_PCS_COMPUTE = 50  # Number of PCs to compute initially

# kNN graph parameters
N_NEIGHBORS = 10  # 🔧 Number of neighbors (increase for smoother manifolds)

# Leiden clustering parameters
CLUSTERING_PARAMS = {
    'resolution': 0.8,  # 🔧 Leiden resolution (determined from previous analysis)
}

# 💡 Tips for tuning:
# - N_PCS: Check PCA elbow plot; typically 20-40 for complex tissues
# - N_NEIGHBORS: 10-15 standard, 20-30 for smoother structure
# - resolution: Lower values (0.2-0.8) for coarse clusters, higher (1.0-2.0) for fine-grained

print("Dimensionality reduction parameters:")
print(f"  N_PCS: {N_PCS}")
print(f"  N_NEIGHBORS: {N_NEIGHBORS}")

print("\nClustering parameters:")
for key, value in CLUSTERING_PARAMS.items():
    print(f"  {key}: {value}")

In [ ]:
# ============================================================================
# CELL TYPE ANNOTATION PARAMETERS
# ============================================================================

ANNOTATION_PARAMS = {
    'label_mode': 'cell',      # 🔧 'cell' for per-cell or 'cluster' for cluster-level
    'margin': 0.05,            # 🔧 Confidence margin for label assignment
    'cluster_agg': 'median',   # 🔧 Aggregation for cluster-level ('median' or 'mean')
}

# Marker gene panel (can be customized)
MARKER_GENES = {
    # General neuron/excitatory
    "Neuron": ["Snap25", "Rbfox3", "Syp"],
    "Excit": ["Slc17a7", "Camk2a", "Satb2"],
    # Excitatory layer-specific markers
    "ExN_L2-4": ["Cux1", "Cux2", "Satb2"],
    "ExN_L5": ["Bcl11b", "Ctip2", "Fezf2"],
    "ExN_L6": ["Tbr1", "Sox5"],
    "ExN_L6b": ["Ctgf"],
    # Inhibitory (generic + subclasses)
    "Inhib": ["Gad1", "Gad2", "Slc6a1"],
    "InN_SST": ["Sst", "Npy", "Chodl"],
    "InN_VIP": ["Vip", "Cck", "Calb2"],
    "InN_PVALB": ["Pvalb", "Gabra1", "Reln"],
    # Glia and vascular
    "Astro": ["Slc1a2", "Slc1a3", "Aqp4", "Aldh1l1", "Gfap"],
    "Oligo": ["Plp1", "Mog", "Mobp", "Mbp"],
    "OPC": ["Pdgfra", "Cspg4", "Sox10"],
    "Micro": ["P2ry12", "Tmem119", "Cx3cr1", "Csf1r", "Sall1", "Aif1"],
    "Endo": ["Pecam1", "Kdr", "Flt1", "Klf2", "Slco1a4"],
    "Peri": ["Pdgfrb", "Rgs5", "Kcnj8", "Abcc9"],
    "VLMC": ["Col1a1", "Col1a2", "Lum", "Dcn"],
    "SMC": ["Acta2", "Myh11", "Tagln"],
}

print("Annotation parameters:")
for key, value in ANNOTATION_PARAMS.items():
    print(f"  {key}: {value}")

print(f"\nNumber of cell type categories: {len(MARKER_GENES)}")

In [ ]:
# ============================================================================
# PARAMETER SUMMARY
# ============================================================================

def display_parameter_summary():
    """Display a formatted summary of all parameters"""
    summary = f"""
    <div style='background-color: #f0f0f0; padding: 15px; border-radius: 5px; font-family: monospace;'>
    <h3 style='margin-top: 0;'>📋 Parameter Summary</h3>
    
    <b>Preset:</b> {PRESETS[SELECTED_PRESET]['name']}<br>
    
    <b>Data:</b><br>
    &nbsp;&nbsp;Samples: {len(SAMPLE_NAMES)}<br>
    &nbsp;&nbsp;Output: {PLOTS_DIR}<br>
    
    <b>QC Filters:</b><br>
    &nbsp;&nbsp;Genes per cell: {CELL_FILTERS['min_genes']}-{CELL_FILTERS['max_genes']}<br>
    &nbsp;&nbsp;Counts per cell: {CELL_FILTERS['min_counts']}-{CELL_FILTERS['max_counts']}<br>
    &nbsp;&nbsp;Max MT%: {CELL_FILTERS['max_mt_pct']}<br>
    &nbsp;&nbsp;Min cells per gene: {GENE_FILTERS['min_cells']}<br>
    
    <b>Doublet Detection:</b><br>
    &nbsp;&nbsp;Expected rate: {DOUBLET_PARAMS['expected_doublet_rate']*100}%<br>
    &nbsp;&nbsp;Manual threshold: {DOUBLET_PARAMS['manual_threshold']}<br>
    
    <b>Clustering:</b><br>
    &nbsp;&nbsp;PCs: {N_PCS}<br>
    &nbsp;&nbsp;Neighbors: {N_NEIGHBORS}<br>
    &nbsp;&nbsp;Resolution: {CLUSTERING_PARAMS['resolution']}<br>
    
    <b>Annotation:</b><br>
    &nbsp;&nbsp;Mode: {ANNOTATION_PARAMS['label_mode']}<br>
    &nbsp;&nbsp;Margin: {ANNOTATION_PARAMS['margin']}<br>
    </div>
    """
    display(HTML(summary))

display_parameter_summary()
print("\n✓ All parameters configured!")

## 3. Stage 1: Data Loading & Integration

Load CellBender-processed data and merge multiple samples.

**Key Parameters:**
- `BASE_PATH`: Path to data directory
- `SAMPLE_NAMES`: List of sample identifiers
- `CUSTOM_NAME`: CellBender output filename pattern

⚠️ **Important**: This uses a custom loading function to properly handle CellBender H5 format.

In [ ]:
def load_cellbender_h5(file_path):
    """Load CellBender processed h5 file
    
    CellBender outputs may have different H5 structure than standard 10x files.
    This function handles the specific format properly, including matrix transposition.
    
    Args:
        file_path: Path to the CellBender H5 file
    
    Returns:
        AnnData object with loaded data (cells × genes)
    """
    with h5py.File(file_path, 'r') as f:
        # Get the matrix data
        matrix = f['matrix']
        features = f['matrix']['features']
        barcodes = f['matrix']['barcodes']
        data = f['matrix']['data']
        indices = f['matrix']['indices']
        indptr = f['matrix']['indptr']
        shape = f['matrix']['shape']
        
        # Read the actual values
        data_vals = data[:]
        indices_vals = indices[:]
        indptr_vals = indptr[:]
        shape_vals = tuple(shape[:])
        
        # Create sparse matrix
        X = sparse.csc_matrix((data_vals, indices_vals, indptr_vals), shape=shape_vals)
        
        # Get feature names and barcodes
        gene_names = [x.decode('utf-8') for x in features['name'][:]]
        gene_ids = [x.decode('utf-8') for x in features['id'][:]]
        cell_barcodes = [x.decode('utf-8') for x in barcodes[:]]
        
        # Create AnnData object (transpose if needed to get cells x genes)
        if X.shape[0] == len(gene_names) and X.shape[1] == len(cell_barcodes):
            # Matrix is genes x cells, transpose to cells x genes
            adata = anndata.AnnData(X.T.tocsr())
        else:
            # Matrix is already cells x genes
            adata = anndata.AnnData(X.tocsr())
        
        adata.var_names = gene_names
        adata.var['gene_ids'] = gene_ids
        adata.obs_names = cell_barcodes
        adata.var_names_make_unique()
    
    return adata


def load_and_merge_cellbender_data(base_path, sample_names, custom_name):
    """Load and merge CellBender H5 files from multiple samples
    
    Args:
        base_path: Base directory path
        sample_names: List of sample identifiers
        custom_name: CellBender filename suffix
    
    Returns:
        Merged AnnData object
    """
    print(f"Loading {len(sample_names)} samples...")
    
    adatas = []
    for sample in sample_names:
        file_path = Path(base_path) / sample / f"{sample}{custom_name}"
        try:
            # Use custom loader for CellBender format
            adata_sample = load_cellbender_h5(file_path)
            
            # Add sample metadata (using orig.ident as in original pipeline)
            adata_sample.obs['sample'] = sample
            adata_sample.obs['orig.ident'] = sample
            
            # Add sample prefix to cell barcodes for uniqueness
            adata_sample.obs_names = [f"{sample}_{barcode}" for barcode in adata_sample.obs_names]
            
            adatas.append(adata_sample)
            print(f"  ✓ {sample}: {adata_sample.n_obs} cells, {adata_sample.n_vars} genes")
        except Exception as e:
            print(f"  ✗ Failed to load {sample}: {e}")
    
    if not adatas:
        raise ValueError("No data loaded! Check your paths.")
    
    # Merge using anndata.concat (join='outer' to keep all genes, fill_value=0 for missing)
    print("\nMerging samples...")
    adata = anndata.concat(adatas, join='outer', fill_value=0)
    adata.var_names_make_unique()
    
    print(f"\n✓ Merged dataset: {adata.n_obs:,} cells × {adata.n_vars:,} genes")
    return adata


def add_metadata(adata, sample_names):
    """Add experimental metadata to AnnData object
    
    ⚠️ IMPORTANT: Customize this function for your experiment!
    The pattern below is specific to the example dataset with 16 samples.
    
    Args:
        adata: AnnData object
        sample_names: List of sample identifiers
    
    Returns:
        AnnData object with added metadata
    """
    print("\nAdding metadata...")
    
    # Example metadata pattern (for 16 samples with specific experimental design)
    # 🔧 CUSTOMIZE THIS FOR YOUR EXPERIMENT!
    if len(sample_names) == 16:
        # Original pipeline pattern: alternating E3/E4, grouped by stimulation, alternating M/F
        metadata = pd.DataFrame({
            'orig.ident': sample_names,
            'Genotype': ['E3', 'E4', 'E3', 'E4'] * 4,
            'Stimulation': ['Ctrl'] * 8 + ['GENUS'] * 8,
            'Sex': ['M', 'M', 'F', 'F'] * 4,
        })
    else:
        # Generic placeholder - YOU MUST CUSTOMIZE THIS
        print("  ⚠️ WARNING: Using placeholder metadata!")
        print("  ⚠️ Edit this function to match your experimental design!")
        metadata = pd.DataFrame({
            'orig.ident': sample_names,
            'Genotype': ['Unknown'] * len(sample_names),
            'Sex': ['Unknown'] * len(sample_names),
            'Stimulation': ['Unknown'] * len(sample_names),
        })
    
    # Map metadata to cells using orig.ident
    for col in ['Genotype', 'Sex', 'Stimulation']:
        adata.obs[col] = adata.obs['orig.ident'].map(
            dict(zip(metadata['orig.ident'], metadata[col]))
        )
    
    print("  ✓ Metadata added")
    print(f"  Metadata columns: {['Genotype', 'Sex', 'Stimulation']}")
    
    return adata

In [ ]:
# Load and merge data
adata = load_and_merge_cellbender_data(BASE_PATH, SAMPLE_NAMES, CUSTOM_NAME)

# Add metadata
adata = add_metadata(adata, SAMPLE_NAMES)

# Display dataset info
print("\n" + "="*50)
print("DATASET SUMMARY")
print("="*50)
print(f"Total cells: {adata.n_obs:,}")
print(f"Total genes: {adata.n_vars:,}")
print(f"Samples: {adata.obs['orig.ident'].nunique()}")
print(f"\nMetadata columns: {list(adata.obs.columns)}")
print(f"\nSample distribution:")
print(adata.obs['orig.ident'].value_counts().sort_index())

## 4. Stage 2: QC Metrics Calculation & Visualization

Calculate quality control metrics and visualize distributions to inform filtering thresholds.

**Key Metrics:**
- `n_genes_by_counts`: Number of genes detected per cell
- `total_counts`: Total UMI counts per cell
- `percent_mt`: Percentage of mitochondrial gene expression
- `percent_ribo`: Percentage of ribosomal gene expression

In [ ]:
def calculate_qc_metrics(adata):
    """Calculate QC metrics for cells
    
    This follows the original pipeline's calculation method.
    
    Args:
        adata: AnnData object
    
    Returns:
        AnnData object with QC metrics added to .obs
    """
    print("Calculating QC metrics...")
    
    # Identify mitochondrial genes
    adata.var['mt'] = adata.var_names.str.startswith(GENE_PATTERNS['mt_pattern'])
    
    # Identify ribosomal genes
    adata.var['ribo'] = adata.var_names.str.match(GENE_PATTERNS['ribo_pattern'])
    
    # Calculate basic QC metrics using scanpy
    sc.pp.calculate_qc_metrics(
        adata,
        percent_top=None,
        log1p=False,
        inplace=True
    )
    
    # Calculate mitochondrial and ribosomal percentages manually
    # (matching original pipeline method for exact reproducibility)
    adata.obs['percent_mt'] = (
        adata[:, adata.var['mt']].X.sum(axis=1).A1 / adata.obs['total_counts']
    ) * 100
    
    adata.obs['percent_ribo'] = (
        adata[:, adata.var['ribo']].X.sum(axis=1).A1 / adata.obs['total_counts']
    ) * 100
    
    print(f"  ✓ Mitochondrial genes: {adata.var['mt'].sum()}")
    print(f"  ✓ Ribosomal genes: {adata.var['ribo'].sum()}")
    print(f"  ✓ QC metrics calculated")
    
    return adata

# Calculate metrics
adata = calculate_qc_metrics(adata)

In [ ]:
def plot_qc_metrics(adata, save_dir=None):
    """Plot QC metric distributions
    
    Args:
        adata: AnnData object with QC metrics
        save_dir: Directory to save plots (optional)
    """
    print("\nPlotting QC metrics...")
    
    # Violin plots
    fig, axes = plt.subplots(1, 4, figsize=(16, 4))
    
    sc.pl.violin(adata, 'n_genes_by_counts', jitter=0.4, ax=axes[0], show=False)
    axes[0].axhline(CELL_FILTERS['min_genes'], color='r', linestyle='--', linewidth=2, label='min')
    axes[0].axhline(CELL_FILTERS['max_genes'], color='r', linestyle='--', linewidth=2, label='max')
    axes[0].legend()
    axes[0].set_title('Genes per cell')
    
    sc.pl.violin(adata, 'total_counts', jitter=0.4, ax=axes[1], show=False)
    axes[1].axhline(CELL_FILTERS['min_counts'], color='r', linestyle='--', linewidth=2, label='min')
    axes[1].axhline(CELL_FILTERS['max_counts'], color='r', linestyle='--', linewidth=2, label='max')
    axes[1].legend()
    axes[1].set_title('Total counts per cell')
    
    sc.pl.violin(adata, 'percent_mt', jitter=0.4, ax=axes[2], show=False)
    axes[2].axhline(CELL_FILTERS['max_mt_pct'], color='r', linestyle='--', linewidth=2, label='max')
    axes[2].legend()
    axes[2].set_title('Mitochondrial %')
    
    sc.pl.violin(adata, 'percent_ribo', jitter=0.4, ax=axes[3], show=False)
    if CELL_FILTERS['max_ribo_pct']:
        axes[3].axhline(CELL_FILTERS['max_ribo_pct'], color='r', linestyle='--', linewidth=2, label='max')
        axes[3].legend()
    axes[3].set_title('Ribosomal %')
    
    plt.tight_layout()
    if save_dir:
        fig.savefig(save_dir / 'qc_violin_plots.png', dpi=300, bbox_inches='tight')
        print(f"  Saved: {save_dir}/qc_violin_plots.png")
    plt.show()
    
    # Scatter plots
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    
    sc.pl.scatter(adata, x='total_counts', y='n_genes_by_counts', ax=axes[0], show=False)
    axes[0].axhline(CELL_FILTERS['min_genes'], color='r', linestyle='--', alpha=0.5)
    axes[0].axhline(CELL_FILTERS['max_genes'], color='r', linestyle='--', alpha=0.5)
    axes[0].axvline(CELL_FILTERS['min_counts'], color='r', linestyle='--', alpha=0.5)
    axes[0].axvline(CELL_FILTERS['max_counts'], color='r', linestyle='--', alpha=0.5)
    axes[0].set_title('Counts vs Genes')
    
    sc.pl.scatter(adata, x='total_counts', y='percent_mt', ax=axes[1], show=False)
    axes[1].axhline(CELL_FILTERS['max_mt_pct'], color='r', linestyle='--', alpha=0.5)
    axes[1].set_title('Counts vs MT%')
    
    plt.tight_layout()
    if save_dir:
        fig.savefig(save_dir / 'qc_scatter_plots.png', dpi=300, bbox_inches='tight')
        print(f"  Saved: {save_dir}/qc_scatter_plots.png")
    plt.show()
    
    # Summary statistics
    print("\nQC Metrics Summary:")
    summary_stats = adata.obs[['n_genes_by_counts', 'total_counts', 'percent_mt', 'percent_ribo']].describe()
    display(summary_stats)

# Plot QC metrics
plot_qc_metrics(adata, save_dir=PLOTS_DIR)

### 💡 Interpreting QC Plots

**What to look for:**

1. **Genes per cell (n_genes_by_counts)**
   - Low values (<200): Likely empty droplets or dead cells
   - Very high values (>8000): Potential doublets
   - Action: Adjust `min_genes` and `max_genes` to capture the main population

2. **Total counts**
   - Should correlate with genes detected
   - Wide spread may indicate batch effects or biological variation
   - Action: Set bounds to exclude extreme outliers

3. **Mitochondrial percentage**
   - High values (>10-20%): Stressed or dying cells
   - Varies by tissue (neurons typically <5%, some tissues naturally higher)
   - Action: Set `max_mt_pct` based on your tissue's characteristics

4. **Scatter plots**
   - Counts vs genes: Should show positive correlation
   - Counts vs MT%: High MT cells often have low counts

**Adjust parameters in Section 2 if needed and re-run from there!**

## 5. Stage 3: Doublet Detection

Identify potential doublets (cells that represent two cells captured together) using Scrublet.

**Key Parameters:**
- `expected_doublet_rate`: Platform-dependent (6-10% typical)
- `manual_threshold`: Score threshold for classification

⚠️ **Critical**: Doublets are detected **per-sample** to account for sample-specific characteristics.

In [ ]:
def detect_doublets_improved(adata, expected_doublet_rate=0.10, manual_threshold=0.35,
                            plot_histograms=True, save_dir=None):
    """Detect doublets using Scrublet with per-sample processing
    
    IMPORTANT: Doublets must be detected per-sample to account for
    sample-specific doublet rates and characteristics.
    
    This follows the original pipeline's implementation exactly.
    
    Args:
        adata: AnnData object (should be after basic QC filtering)
        expected_doublet_rate: Expected doublet rate (default 0.10)
        manual_threshold: Manual score threshold (default 0.35)
        plot_histograms: Whether to plot per-sample histograms
        save_dir: Directory to save plots
    
    Returns:
        AnnData object with doublet_score and predicted_doublet columns
    """
    print("\nRunning doublet detection (per-sample)...")
    print(f"  Expected doublet rate: {expected_doublet_rate*100}%")
    print(f"  Manual threshold: {manual_threshold}")
    
    # Initialize arrays to store results for all cells
    all_scores = np.zeros(adata.n_obs)
    all_predictions = np.zeros(adata.n_obs, dtype=bool)
    
    # Get unique samples
    samples = adata.obs['orig.ident'].unique()
    print(f"  Processing {len(samples)} samples separately\n")
    
    # Setup plot if requested
    if plot_histograms and save_dir:
        n_rows = (len(samples) + 3) // 4  # 4 columns
        fig, axes = plt.subplots(n_rows, 4, figsize=(16, 3*n_rows))
        axes = axes.flatten() if len(samples) > 1 else [axes]
    
    # Process each sample separately
    for idx, sample in enumerate(samples):
        print(f"Sample {idx+1}/{len(samples)}: {sample}")
        
        # Get sample mask and indices
        mask = adata.obs['orig.ident'] == sample
        sample_indices = np.where(mask)[0]
        
        # Extract sample data (MUST use .copy()!)
        adata_sample = adata[mask].copy()
        
        # Skip if too few cells
        if adata_sample.n_obs < 100:
            print(f"  ⚠️  Skipping - only {adata_sample.n_obs} cells\n")
            continue
        
        # Initialize Scrublet for this sample
        scrub = scr.Scrublet(
            adata_sample.X,
            expected_doublet_rate=expected_doublet_rate
        )
        
        # Run doublet detection
        doublet_scores, predicted_doublets = scrub.scrub_doublets(
            min_counts=DOUBLET_PARAMS['min_counts'],
            min_cells=DOUBLET_PARAMS['min_cells'],
            min_gene_variability_pctl=DOUBLET_PARAMS['min_gene_variability_pctl'],
            n_prin_comps=DOUBLET_PARAMS['n_prin_comps'],
            verbose=False,
        )
        
        # Get automatic threshold
        auto_threshold = scrub.call_doublets(threshold=None)[1]
        
        # Use manual threshold if specified, otherwise use automatic
        if manual_threshold is not None:
            threshold = manual_threshold
            predicted_doublets = doublet_scores > threshold
        else:
            threshold = auto_threshold
        
        # Cap threshold to avoid missing obvious doublets
        if threshold > 0.4:
            print(f"  ⚠️  High auto threshold {threshold:.2f}, capping at 0.4")
            threshold = 0.4
            predicted_doublets = doublet_scores > threshold
        
        # Store results for this sample's cells
        all_scores[sample_indices] = doublet_scores
        all_predictions[sample_indices] = predicted_doublets
        
        # Calculate statistics
        n_doublets = predicted_doublets.sum()
        pct_doublets = n_doublets / len(doublet_scores) * 100
        
        print(f"  Cells: {len(doublet_scores):,}")
        print(f"  Threshold: {threshold:.3f} (auto: {auto_threshold:.3f})")
        print(f"  Doublets: {n_doublets:,} ({pct_doublets:.1f}%)")
        print(f"  Score range: [{doublet_scores.min():.3f}, {doublet_scores.max():.3f}]\n")
        
        # Plot histogram for this sample
        if plot_histograms and save_dir and idx < len(axes):
            ax = axes[idx]
            ax.hist(doublet_scores, bins=50, alpha=0.7, edgecolor='black', color='steelblue')
            ax.axvline(threshold, color='red', linestyle='--', linewidth=2,
                      label=f'Threshold: {threshold:.2f}')
            ax.set_title(f"{sample}\n{n_doublets} doublets ({pct_doublets:.1f}%)", fontsize=10)
            ax.set_xlabel('Doublet Score', fontsize=9)
            ax.set_ylabel('Frequency', fontsize=9)
            ax.legend(fontsize=8)
            ax.grid(alpha=0.3)
    
    # Hide unused subplots
    if plot_histograms and save_dir:
        for idx in range(len(samples), len(axes)):
            axes[idx].axis('off')
        
        plt.tight_layout()
        plt.savefig(save_dir / 'doublet_score_histograms.png', dpi=300, bbox_inches='tight')
        print(f"✓ Saved: {save_dir}/doublet_score_histograms.png")
        plt.show()
        plt.close()
    
    # Add results to adata
    adata.obs['doublet_score'] = all_scores
    adata.obs['predicted_doublet'] = all_predictions
    
    # Overall summary
    total_doublets = all_predictions.sum()
    overall_rate = total_doublets / len(all_predictions) * 100
    
    print("="*60)
    print(f"OVERALL SUMMARY")
    print("="*60)
    print(f"Total cells processed: {len(all_predictions):,}")
    print(f"Total doublets detected: {total_doublets:,} ({overall_rate:.1f}%)")
    print("="*60)
    
    return adata

In [ ]:
# Apply basic QC filters BEFORE doublet detection
# This is critical - we filter cells first to improve doublet detection
print("\n" + "="*60)
print("PREPARING CELLS FOR DOUBLET DETECTION")
print("="*60)
print("Applying initial QC filters for doublet detection...")
print(f"Starting with {adata.n_obs:,} cells\n")

adata_for_doublets = adata[
    (adata.obs.n_genes_by_counts >= CELL_FILTERS['min_genes']) &
    (adata.obs.n_genes_by_counts <= CELL_FILTERS['max_genes']) &
    (adata.obs.percent_mt <= CELL_FILTERS['max_mt_pct'])
].copy()

print(f"Cells passing initial QC: {adata_for_doublets.n_obs:,}")
print(f"Cells filtered out: {adata.n_obs - adata_for_doublets.n_obs:,}\n")

# Detect doublets on QC-filtered cells
adata_for_doublets = detect_doublets_improved(
    adata_for_doublets,
    expected_doublet_rate=DOUBLET_PARAMS['expected_doublet_rate'],
    manual_threshold=DOUBLET_PARAMS['manual_threshold'],
    plot_histograms=True,
    save_dir=PLOTS_DIR,
)

# Transfer doublet annotations back to original adata
print("\nTransferring doublet annotations to full dataset...")
adata.obs['doublet_score'] = 0.0
adata.obs['predicted_doublet'] = False
adata.obs.loc[adata_for_doublets.obs.index, 'doublet_score'] = adata_for_doublets.obs['doublet_score']
adata.obs.loc[adata_for_doublets.obs.index, 'predicted_doublet'] = adata_for_doublets.obs['predicted_doublet']

print("\n✓ Doublet detection complete!")

### 💡 Tuning Doublet Detection

**Key considerations:**

1. **Expected doublet rate**
   - 10x Chromium v2: ~4-6%
   - 10x Chromium v3: ~6-8%
   - High-throughput: ~8-10%
   - Check your platform specifications

2. **Manual threshold**
   - Default: 0.35
   - Lower (0.25-0.30): More stringent, removes more cells
   - Higher (0.40-0.50): More permissive, retains more cells
   - Examine histograms: clear separation = good, overlap = difficult

3. **Per-sample processing**
   - ⚠️ **Critical**: Each sample has unique characteristics
   - Different cell loading densities → different doublet rates
   - Different cell types → different score distributions
   - Always process samples separately!

4. **What to check after clustering:**
   - Doublets should appear as intermediate clusters on UMAP
   - Check if "doublet clusters" express markers from 2+ cell types
   - If residual doublets remain, lower the threshold and re-run

## 6. Stage 4: Cell & Gene Filtering

Apply QC filters to remove low-quality cells and rarely-expressed genes.

**Filters Applied (in order):**
1. Remove cells with too few genes (min_genes)
2. Remove genes expressed in too few cells (min_cells)
3. Remove cells with too many genes (max_genes)
4. Remove cells with high MT% (max_mt_pct)
5. Remove cells outside count ranges (min/max_counts)
6. Remove cells with high ribosomal% (if set)
7. **Remove doublets LAST**

⚠️ **Important**: Doublets are removed LAST after all other QC filters.

In [ ]:
def filter_cells_and_genes(adata, min_genes=200, max_genes=8000, max_mt_pct=10,
                          min_counts=1000, max_counts=50000, max_ribo_pct=None):
    """Apply QC filtering in the correct order
    
    IMPORTANT: Filtering order matters!
    This follows the original pipeline's order exactly:
    1. min_genes → 2. min_cells → 3. max_genes → 4. max_mt_pct →
    5. count filters → 6. ribo% → 7. doublets LAST
    
    Args:
        adata: AnnData object
        min_genes: Minimum genes per cell
        max_genes: Maximum genes per cell
        max_mt_pct: Maximum mitochondrial percentage
        min_counts: Minimum total counts per cell
        max_counts: Maximum total counts per cell
        max_ribo_pct: Maximum ribosomal percentage (optional)
    
    Returns:
        Filtered AnnData object
    """
    print("\n" + "="*60)
    print("APPLYING QC FILTERS")
    print("="*60)
    print(f"Starting: {adata.n_obs:,} cells × {adata.n_vars:,} genes\n")
    
    # 1. Filter cells by minimum genes
    n_before = adata.n_obs
    sc.pp.filter_cells(adata, min_genes=min_genes)
    print(f"[1/7] After min_genes ({min_genes}) filter:")
    print(f"      {adata.n_obs:,} cells ({n_before - adata.n_obs:,} removed)\n")
    
    # 2. Filter genes by minimum cells expressing
    n_genes_before = adata.n_vars
    sc.pp.filter_genes(adata, min_cells=GENE_FILTERS['min_cells'])
    print(f"[2/7] After min_cells ({GENE_FILTERS['min_cells']}) filter:")
    print(f"      {adata.n_vars:,} genes ({n_genes_before - adata.n_vars:,} removed)\n")
    
    # 3. Filter cells by maximum genes
    n_before = adata.n_obs
    adata = adata[adata.obs.n_genes_by_counts < max_genes].copy()
    print(f"[3/7] After max_genes ({max_genes}) filter:")
    print(f"      {adata.n_obs:,} cells ({n_before - adata.n_obs:,} removed)\n")
    
    # 4. Filter cells by MT percentage
    n_before = adata.n_obs
    adata = adata[adata.obs.percent_mt < max_mt_pct].copy()
    print(f"[4/7] After max_mt_pct ({max_mt_pct}%) filter:")
    print(f"      {adata.n_obs:,} cells ({n_before - adata.n_obs:,} removed)\n")
    
    # 5a. Optional: Filter by minimum counts
    if min_counts is not None:
        n_before = adata.n_obs
        adata = adata[adata.obs.total_counts >= min_counts].copy()
        print(f"[5a/7] After min_counts ({min_counts}) filter:")
        print(f"       {adata.n_obs:,} cells ({n_before - adata.n_obs:,} removed)\n")
    else:
        print(f"[5a/7] min_counts filter: SKIPPED (not set)\n")
    
    # 5b. Optional: Filter by maximum counts
    if max_counts is not None:
        n_before = adata.n_obs
        adata = adata[adata.obs.total_counts <= max_counts].copy()
        print(f"[5b/7] After max_counts ({max_counts}) filter:")
        print(f"       {adata.n_obs:,} cells ({n_before - adata.n_obs:,} removed)\n")
    else:
        print(f"[5b/7] max_counts filter: SKIPPED (not set)\n")
    
    # 6. Optional: Filter by ribosomal percentage
    if max_ribo_pct is not None:
        n_before = adata.n_obs
        adata = adata[adata.obs.percent_ribo < max_ribo_pct].copy()
        print(f"[6/7] After max_ribo_pct ({max_ribo_pct}%) filter:")
        print(f"      {adata.n_obs:,} cells ({n_before - adata.n_obs:,} removed)\n")
    else:
        print(f"[6/7] max_ribo_pct filter: SKIPPED (not set)\n")
    
    # 7. Remove doublets LAST (most important!)
    n_before = adata.n_obs
    n_doublets = adata.obs.predicted_doublet.sum()
    adata = adata[~adata.obs.predicted_doublet].copy()
    print(f"[7/7] After doublet removal:")
    print(f"      {adata.n_obs:,} cells ({n_doublets:,} doublets removed)\n")
    
    print("="*60)
    print(f"FINAL: {adata.n_obs:,} cells × {adata.n_vars:,} genes")
    print("="*60)
    
    return adata

# Apply filters
adata = filter_cells_and_genes(
    adata,
    min_genes=CELL_FILTERS['min_genes'],
    max_genes=CELL_FILTERS['max_genes'],
    max_mt_pct=CELL_FILTERS['max_mt_pct'],
    min_counts=CELL_FILTERS['min_counts'],
    max_counts=CELL_FILTERS['max_counts'],
    max_ribo_pct=CELL_FILTERS['max_ribo_pct'],
)

In [ ]:
# Visualize filtered data statistics
print("\nGenerating filtered data summary plots...")

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Cells per sample
sample_counts = adata.obs['orig.ident'].value_counts().sort_index()
axes[0, 0].bar(range(len(sample_counts)), sample_counts.values, color='steelblue')
axes[0, 0].set_xticks(range(len(sample_counts)))
axes[0, 0].set_xticklabels(sample_counts.index, rotation=45, ha='right', fontsize=8)
axes[0, 0].set_xlabel('Sample')
axes[0, 0].set_ylabel('Number of cells')
axes[0, 0].set_title('Cells per sample (after filtering)')
axes[0, 0].grid(axis='y', alpha=0.3)

# Gene count distribution
axes[0, 1].hist(adata.obs['n_genes_by_counts'], bins=50, alpha=0.7, color='coral', edgecolor='black')
axes[0, 1].set_xlabel('Genes per cell')
axes[0, 1].set_ylabel('Frequency')
axes[0, 1].set_title('Gene count distribution (filtered)')
axes[0, 1].grid(alpha=0.3)

# MT% distribution
axes[1, 0].hist(adata.obs['percent_mt'], bins=50, alpha=0.7, color='orange', edgecolor='black')
axes[1, 0].set_xlabel('MT %')
axes[1, 0].set_ylabel('Frequency')
axes[1, 0].set_title('MT% distribution (filtered)')
axes[1, 0].grid(alpha=0.3)

# UMI count distribution
axes[1, 1].hist(adata.obs['total_counts'], bins=50, alpha=0.7, color='lightgreen', edgecolor='black')
axes[1, 1].set_xlabel('Total UMI counts')
axes[1, 1].set_ylabel('Frequency')
axes[1, 1].set_title('UMI count distribution (filtered)')
axes[1, 1].grid(alpha=0.3)

plt.tight_layout()
plt.savefig(PLOTS_DIR / 'filtered_data_summary.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"\n✓ Filtered data summary saved to {PLOTS_DIR}/filtered_data_summary.png")

### 🎛️ Parameter Tuning: QC Results

Review the plots above. What did you observe?

<details>
<summary>📊 Two distinct populations in genes per cell plot</summary>

**Diagnosis:** Low population likely empty droplets/dead cells

**Action:**
```python
# Scroll back to Parameter Configuration and update:
CELL_FILTERS['min_genes'] = 300  # Increase threshold
```
**Then:** Re-run from Stage 2
</details>

<details>
<summary>📊 MT% tail extends to 20%+</summary>

**Diagnosis:** Stressed/dying cells in tail

**Action:**
```python
CELL_FILTERS['max_mt_pct'] = 10  # or 5 for neurons
```
**Then:** Re-run from Stage 2
</details>

<details>
<summary>📊 Metrics look good (tight distributions)</summary>

**Diagnosis:** ✅ High quality data

**Action:** Proceed to Stage 3
</details>

**See `../PARAMETER_TUNING_GUIDE.md` for 35+ scenarios**


In [ ]:
# QC Metrics Assessment
print("\n" + "="*60)
print("QC METRICS ASSESSMENT")
print("="*60)

median_genes = adata.obs['n_genes_by_counts'].median()
median_mt = adata.obs['percent_mt'].median()

print(f"Median genes/cell: {median_genes:.0f}")
if median_genes < 2000:
    print("⚠️  LOW: Check sequencing depth")
elif median_genes > 5000:
    print("✅ EXCELLENT: High quality")
else:
    print("✅ GOOD: Normal range")

print(f"\nMedian MT%: {median_mt:.2f}%")
if median_mt > 10:
    print("⚠️  HIGH: Consider max_mt_pct = 8")
elif median_mt < 5:
    print("✅ EXCELLENT: Low stress")
else:
    print("✅ GOOD: Acceptable")

print("\n💡 NEXT STEPS:")
if median_genes < 2000 or median_mt > 10:
    print("   • Adjust parameters and re-run Stage 2")
else:
    print("   • Proceed to Stage 3")


### 🎛️ Parameter Tuning: Doublet Detection

Review the per-sample histograms above.

<details>
<summary>📊 Doublet rates 6-10% per sample</summary>

**Diagnosis:** ✅ Normal for 10x

**Action:** Proceed to Stage 4
</details>

<details>
<summary>📊 One or more samples >15% doublets</summary>

**Diagnosis:** Sample overload or quality issue

**Action:**
```python
# If unexpected:
DOUBLET_PARAMS['manual_threshold'] = 0.30  # More stringent
```
**Then:** Re-run from Stage 3
</details>

<details>
<summary>📊 All samples <3% doublets</summary>

**Diagnosis:** ⚠️ Threshold may be too permissive

**Action:**
```python
DOUBLET_PARAMS['manual_threshold'] = 0.25  # Lower
```
**Then:** Re-run from Stage 3
</details>


In [ ]:
# Doublet Detection Summary
print("\n" + "="*60)
print("DOUBLET SUMMARY")
print("="*60)

summary = adata.obs.groupby('orig.ident').agg({
    'predicted_doublet': ['count', 'sum']
})
summary.columns = ['n_cells', 'n_doublets']
summary['pct'] = (summary['n_doublets'] / summary['n_cells'] * 100).round(1)

print("\nPer-sample:")
print(summary)

overall = (adata.obs['predicted_doublet'].sum() / len(adata.obs)) * 100
print(f"\nOverall: {overall:.1f}%")

if overall < 3:
    print("⚠️  Very low - consider threshold = 0.25")
elif overall > 20:
    print("⚠️  Very high - investigate quality")
elif 6 <= overall <= 10:
    print("✅ Expected range")
else:
    print("✅ Acceptable")


### 🎛️ Parameter Tuning: Filtering Results

Check cell retention above.

<details>
<summary>📊 <5,000 cells retained</summary>

**Diagnosis:** May lack power for clustering

**Action:** Review if filters too stringent
```python
CELL_FILTERS['min_genes'] = 150  # Relax
CELL_FILTERS['max_mt_pct'] = 12
```
**Then:** Re-run from Stage 2
</details>

<details>
<summary>📊 5,000-50,000 cells retained</summary>

**Diagnosis:** ✅ Sufficient

**Action:** Proceed to save and Notebook 2
</details>

<details>
<summary>📊 One sample has <500 cells</summary>

**Diagnosis:** Sample-specific issue

**Action:** Investigate that sample's QC metrics
</details>


In [ ]:
# Filtering Assessment
print("\n" + "="*60)
print("FILTERING ASSESSMENT")
print("="*60)

print(f"Total cells: {adata.n_obs:,}")
print(f"Total genes: {adata.n_vars:,}")

sample_counts = adata.obs['orig.ident'].value_counts()
print(f"\nCells per sample:")
print(f"  Min: {sample_counts.min():,}")
print(f"  Max: {sample_counts.max():,}")
print(f"  Mean: {sample_counts.mean():.0f}")

if adata.n_obs < 5000:
    print("\n⚠️  <5k cells - may need to relax filters")
elif adata.n_obs > 50000:
    print("\n✅ >50k cells - excellent power!")
else:
    print("\n✅ Sufficient cells for analysis")

if sample_counts.min() < 500:
    low_samples = sample_counts[sample_counts < 500].index.tolist()
    print(f"\n⚠️  Low-count samples: {low_samples}")
    print("   → Investigate these samples")


## Save Output

Save QC-filtered data for Notebook 2.

In [ ]:
# Save QC-filtered data
output_file = 'outputs/qc_filtered_data.h5ad'
Path('outputs').mkdir(exist_ok=True)

# Store parameters used
adata.uns['pipeline_params'] = {
    'notebook': '1_setup_qc_filtering',
    'cell_filters': CELL_FILTERS,
    'doublet_params': DOUBLET_PARAMS,
    'gene_patterns': GENE_PATTERNS,
    'gene_filters': GENE_FILTERS,
}

adata.write(output_file)

print("\n" + "="*60)
print("NOTEBOOK 1 COMPLETE")
print("="*60)
print(f"✓ Saved: {output_file}")
print(f"  Cells: {adata.n_obs:,}")
print(f"  Genes: {adata.n_vars:,}")
print(f"  Size: {Path(output_file).stat().st_size / 1e6:.1f} MB")
print("\n➡️  NEXT: Open 2_clustering_markers.ipynb")
